In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4' 

import deepchem as dc
import pandas as pd
import pickle
import numpy as np
import warnings
from sklearn.ensemble import RandomForestRegressor

from utils import *
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt

def runExp(params, smiles, expt, feat, part, model='graph'):#partition (train, test, val)
    smiles = partition(smiles, part)
    expt = partition(expt, part)
    feat = partition(feat, part)
    # if(model=='graph'):
    featurizer = dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)
    # else:
        # featurizer = dc.feat.CircularFingerprint()
        
    train = dc.data.NumpyDataset(X=featurizer.featurize(smiles[0]), y=np.array(expt[0]-feat[0]).transpose())
    np.random.seed(10)
    # if(model=='graph'):
    model = dc.models.GraphConvModel(n_tasks=1, graph_conv_layers=params['graph_conv_layers'],
                                         mode='regression', dropout=params['dropout'], 
                                         batch_normalize=params['batch_normalize'], 
                                         batch_size=params['batch_size'], 
                                         dense_layer_size=params['dense_layer_size'])
    # else:
        # sklearn_model = RandomForestRegressor(n_jobs=-1, n_estimators=500,max_depth=100,
        #                                      min_samples_leaf=3)
        # model = dc.models.SklearnModel(sklearn_model)
    #x = []
    #for i in range(params['epochs']):
    ##    #model.fit(train, nb_epoch=1)
    #    x.append(model.fit(train, nb_epoch=1))
    #    print(x)
    #plt.plot(x, np.arange(len(x)))
    #plt.show()
    # if(model=='graph'):
    model.fit(train, nb_epoch=params['epochs'])
    # else:
        # model.fit(train)

    p = ()

    for i in range(len(part)):
        p += (np.array(model.predict_on_batch(featurizer.featurize(smiles[i])).flatten()),)

    p_true = {'test' : list(expt[1]), 'train' : list(expt[0])}
    p_phy = {'test' : list(feat[1]), 'train' : list(feat[0])}
    p_corr = {'test' : list(p[1] + feat[1]), 'train' : list(p[0] + feat[0])}

    if(len(part) > 2):
        p_true['valid'] = list(expt[2])
        p_phy['valid'] = list(feat[2])
        p_corr['valid'] = list(p[2]+feat[2])

    return p_true, p_phy, p_corr

def kfold(params, b = None, val = None):
    k = params['kfold']
    if(k == -1):
        return kfinal(params, b=b, val=val)

    if(type(b) == type(None)):
        b = psuedoScramble(expt, bins=int(len(expt)/k))
    

    folds = []
    for i in range(k):
        folds.append(b[i::k])
    
    p_true = []
    p_phy = []
    p_corr = []
    stats = []
    for i in range(k):
        part = (np.hstack(tuple(folds[:i])+tuple(folds[(i+1):])), folds[i])#train, test, val
        
        if(type(val) != type(None)):
            part += (val,)
        #    print("(kth, train, test, val)  :  ", (i, len(part[0]), len(part[1]), len(part[2])))
        #else:
        #    print("(kth, train, test)  :  ", (i, len(part[0]), len(part[1])))
        
        
        pt, pp, pc = runExp(params, smiles, expt, feats[params['feat']], part)

        p_true.append(pt)
        p_phy.append(pp)
        p_corr.append(pc)
       

        if(type(val) != type(None)):
             stats.append({
                'phy_rmsd' :     {'test' : rmsd(        pt['test'], pp['test']), 'train' : rmsd(        pt['train'], pp['train']), 'valid' : rmsd(       pt['valid'], pp['valid'])},
                'ml_rmsd'  :     {'test' : rmsd(        pt['test'], pc['test']), 'train' : rmsd(        pt['train'], pc['train']), 'valid' : rmsd(       pt['valid'], pc['valid'])},
                'phy_md' :       {'test' : md(          pt['test'], pp['test']), 'train' : md(          pt['train'], pp['train']), 'valid' : md(         pt['valid'], pp['valid'])},
                'ml_md'  :       {'test' : md(          pt['test'], pc['test']), 'train' : md(          pt['train'], pc['train']), 'valid' : md(         pt['valid'], pc['valid'])},
                'phy_out_rmsd' : {'test' : ormsd(0.05,  pt['test'], pp['test']), 'train' : ormsd(0.05,  pt['train'], pp['train']), 'valid' : ormsd(0.05, pt['valid'], pp['valid'])},
                'ml_out_rmsd'  : {'test' : ormsd(0.05,  pt['test'], pc['test']), 'train' : ormsd(0.05,  pt['train'], pc['train']), 'valid' : ormsd(0.05, pt['valid'], pc['valid'])}
            })#
        else:
             stats.append({
                'phy_rmsd' :     {'test' : rmsd(        pt['test'], pp['test']), 'train' : rmsd(        pt['train'], pp['train'])},
                'ml_rmsd'  :     {'test' : rmsd(        pt['test'], pc['test']), 'train' : rmsd(        pt['train'], pc['train'])},
                'phy_md' :       {'test' : md(          pt['test'], pp['test']), 'train' : md(          pt['train'], pp['train'])},
                'ml_md'  :       {'test' : md(          pt['test'], pc['test']), 'train' : md(          pt['train'], pc['train'])},
                'phy_out_rmsd' : {'test' : ormsd(0.05,  pt['test'], pp['test']), 'train' : ormsd(0.05,  pt['train'], pp['train'])},
                'ml_out_rmsd'  : {'test' : ormsd(0.05,  pt['test'], pc['test']), 'train' : ormsd(0.05,  pt['train'], pc['train'])}
            })#

    return p_true, p_phy, p_corr, stats

def kfinal(params, b = None, val = None):

    if(type(b) == type(None)):
        b = psuedoScramble(expt, bins=int(len(expt)/k))

    
    stats = []
    
    part = (b, val)
    p_true, p_phy, p_corr = runExp(params, smiles, expt, feats[params['feat']], part)

    
    stats.append({
        'phy_rmsd' :     {'test' : rmsd(        p_true['test'], p_phy ['test']), 'train' : rmsd(        p_true['train'], p_phy ['train'])},
        'ml_rmsd'  :     {'test' : rmsd(        p_true['test'], p_corr['test']), 'train' : rmsd(        p_true['train'], p_corr['train'])},
        'phy_md' :       {'test' : md(          p_true['test'], p_phy ['test']), 'train' : md(          p_true['train'], p_phy ['train'])},
        'ml_md'  :       {'test' : md(          p_true['test'], p_corr['test']), 'train' : md(          p_true['train'], p_corr['train'])},
        'phy_out_rmsd' : {'test' : ormsd(0.05,  p_true['test'], p_phy ['test']), 'train' : ormsd(0.05,  p_true['train'], p_phy ['train'])},
        'ml_out_rmsd'  : {'test' : ormsd(0.05,  p_true['test'], p_corr['test']), 'train' : ormsd(0.05,  p_true['train'], p_corr['train'])}
    })#

    return [p_true], [p_phy], [p_corr], stats

In [2]:
freeSolve = pickle.load(open('dicts/consol.pickle', 'rb')) # FreeSolve Database

expt, tip, smiles, gbn, igb, asc, zap, cha, nul = [], [], [], [], [], [], [], [], []
for i in freeSolve.keys():
    expt.append(freeSolve[i]['expt'])
    smiles.append(freeSolve[i]['smiles'])
    #
    tip.append(freeSolve[i]['calc'])
    gbn.append(freeSolve[i]['gbnsr6'])
    igb.append(freeSolve[i]['igb5'])
    asc.append(freeSolve[i]['asc'])
    zap.append(freeSolve[i]['zap9'])
    cha.append(freeSolve[i]['cha'])
    nul.append(0)
feats = {'tip3p' : tip, 'gbnsr6' : gbn, 'igb5' : igb, 'asc' : asc, 'null' : nul, 'zap9' : zap, 'cha' : cha}

In [3]:
np.random.seed(10)
b = list(psuedoScramble(expt, bins=int(len(expt)/10)))

val = []
for i in range(len(b)//8):
    #j = np.random.randint(0, len(b))
    j = i*7
    val.append(b.pop(j))
np.mean(b)
len(b)

562

In [4]:
params = [{'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'tip3p', 
           'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]}#,
         # {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'gbnsr6', 
         #  'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},
         # {'epochs' : 500, 'dropout' : 0.6, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'asc', 
         #  'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},
         # {'epochs' : 500, 'dropout' : 0.6, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'igb5', 
         #  'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},
         # {'epochs' : 500, 'dropout' : 0.6, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'zap9', 
         #  'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},          
         # {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'cha', 
         #  'kfold' : -1, 'dense_layer_size' : 27, 'graph_conv_layers' : [32, 32]},
         # {'epochs' : 500, 'dropout' : 0.4, 'batch_normalize' : False, 'batch_size' : 100, 'feat' : 'null', 
         #  'kfold' : -1, 'dense_layer_size' : 100, 'graph_conv_layers' : [150, 150]}
         ]

In [5]:
%%time
iter = 1
error = 0.2
for p in params:
    np.random.seed(10)
    # print(p)
    print(p['feat'])
    p_true, p_phy, p_corr, stats = [], [], [], []
    for i in range(iter):
        pt, pp, pc, ps = kfold(p, b, val)
        p_true += pt
        p_phy += pp
        p_corr += pc
        stats += ps
    p_test, p_train, ml_test, ml_train,test_means,train_means = [],[],[],[],[],[]
    
    p_true_test = p_true[0]['test']
    p_true_train = p_true[0]['train']
    test,train=np.zeros((len(p_true_test), iter)),np.zeros((len(p_true_train), iter))
    for i in range(iter):
        p_test.append(stats[i]['phy_rmsd']['test'])
        p_train.append(stats[i]['phy_rmsd']['train'])
        ml_test.append(stats[i]['ml_rmsd']['test'])
        ml_train.append(stats[i]['ml_rmsd']['train'])
        for j in range(len(p_true_test)):
            test[j][i] = p_corr[i]['test'][j]
        for j in range(len(p_true_train)):
            train[j][i] = p_corr[i]['train'][j]
    for j in range(len(p_true_test)):
        test_means.append(np.mean(test[j]))
    for j in range(len(p_true_train)):
        train_means.append(np.mean(train[j]))
                           
    print('physics model: test',np.round(np.mean(p_test),3),'±',np.round(error*np.std(p_test),3),
              'train',np.round(np.mean(p_train),3),'±',np.round(error*np.std(p_train),3))
    print('physics + ml: test',np.round(np.mean(ml_test),3),'±',np.round(error*np.std(ml_test),3),
              'train',np.round(np.mean(ml_train),3),'±',np.round(error*np.std(ml_train),3))
    print('ensemble ml: test',np.round(rmsd(p_true_test,test_means),3),
              'train',np.round(rmsd(p_true_train,train_means),3))
    # print('physics model: test',stats[0]['phy_rmsd']['test'],'train',stats[0]['phy_rmsd']['train'])
    # print('physics + ml: test',stats[0]['ml_rmsd']['test'],'train',stats[0]['ml_rmsd']['train'])
    print()

tip3p
physics model: test 1.435 ± 0.0 train 1.556 ± 0.0
physics + ml: test 0.87 ± 0.0 train 0.312 ± 0.0
ensemble ml: test 0.87 train 0.312

null
physics model: test 5.45 ± 0.0 train 5.402 ± 0.0
physics + ml: test 1.15 ± 0.0 train 0.443 ± 0.0
ensemble ml: test 1.15 train 0.443

CPU times: user 3min 13s, sys: 19.8 s, total: 3min 33s
Wall time: 54.8 s


In [ ]:
# 100 iterations
# tip3p
# physics model: test 1.435 ± 0.0 train 1.556 ± 0.0
# physics + ml: test 1.025 ± 0.048 train 0.695 ± 0.032
# ensemble ml: test 0.986 train 0.657

# gbnsr6
# physics model: test 1.855 ± 0.0 train 1.64 ± 0.0
# physics + ml: test 1.645 ± 0.042 train 1.075 ± 0.035
# ensemble ml: test 1.62 train 1.034

# cha
# physics model: test 1.315 ± 0.0 train 1.77 ± 0.0
# physics + ml: test 1.075 ± 0.037 train 1.132 ± 0.042
# ensemble ml: test 1.036 train 1.092

# null
# physics model: test 5.45 ± 0.0 train 5.402 ± 0.0
# physics + ml: test 1.652 ± 0.085 train 1.147 ± 0.077
# ensemble ml: test 1.598 train 1.093

# CPU times: user 3h 38min 33s, sys: 36min 27s, total: 4h 15min 1s
# Wall time: 1h 41min 2s



# tip3p
# physics model: test 1.4352081556345755 train 1.5560596697201357
# physics + ml: test 1.1102706751387692 train 0.7603896695181649

# gbnsr6
# physics model: test 1.8545443800079835 train 1.6401879965014992
# physics + ml: test 1.5278740925678005 train 0.7749499295595876

# asc
# physics model: test 2.6515044880218475 train 2.489083699259972
# physics + ml: test 1.4313291924776648 train 1.023530822279272

# igb5
# physics model: test 3.1124919618370104 train 2.804571287833655
# physics + ml: test 1.5310707358788274 train 0.9549467028047232

# zap9
# physics model: test 1.854544509594727 train 1.6401880873566563
# physics + ml: test 1.5271195777731705 train 0.7036319203811187

# cha
# physics model: test 1.3152918288589681 train 1.7703504244090553
# physics + ml: test 1.0098561587190191 train 0.7673784783591486

# null
# physics model: test 5.44957314842181 train 5.4019556726215345
# physics + ml: test 1.733922182659022 train 1.2114719143098274



# tip3p
# physics model: test 1.4352081556345755 train 1.5560596697201357
# physics + ml: test 1.0792134408802871 train 0.7136749042884051

# gbnsr6
# physics model: test 1.8545443800079835 train 1.6401879965014992
# physics + ml: test 1.342211826762011 train 0.7337795399874788

# asc
# physics model: test 2.6515044880218475 train 2.489083699259972
# physics + ml: test 1.556290543899444 train 1.0363204752175816

# igb5
# physics model: test 3.1124919618370104 train 2.804571287833655
# physics + ml: test 1.3207559016435333 train 0.9926163530590447

# zap9
# physics model: test 1.854544509594727 train 1.6401880873566563
# physics + ml: test 1.3775918973869625 train 0.7288532312340192

# cha
# physics model: test 1.3152918288589681 train 1.7703504244090553
# physics + ml: test 0.8880298471535839 train 0.7577906900601187

# null
# physics model: test 5.44957314842181 train 5.4019556726215345
# physics + ml: test 1.6704634292515732 train 1.1299400958947197


# tip3p
# physics model: test 1.4352081556345755 train 1.5560596697201357
# physics + ml: test 1.0087192364547357 train 0.6929096072991768
# gbnsr6
# physics model: test 1.8545443800079835 train 1.6401879965014992
# physics + ml: test 1.5621380205712474 train 0.8214577818349262
# asc
# physics model: test 2.6515044880218475 train 2.489083699259972
# physics + ml: test 1.7419858422878387 train 0.9718657713636776
# igb5
# physics model: test 3.1124919618370104 train 2.804571287833655
# physics + ml: test 1.8339452589669654 train 1.1062774171937182
# zap9
# physics model: test 1.854544509594727 train 1.6401880873566563
# physics + ml: test 1.4442621026863973 train 0.7413050305145663
# cha
# physics model: test 1.3152918288589681 train 1.7703504244090553
# physics + ml: test 0.9641223950581289 train 0.7932160691476784
# null
# physics model: test 5.44957314842181 train 5.4019556726215345
# physics + ml: test 1.623001677487049 train 1.1344209698457504

# {'epochs': 500, 'dropout': 0.4, 'batch_normalize': False, 'batch_size': 100, 'feat': 'tip3p', 'kfold': -1, 'dense_layer_size': 27, 'graph_conv_layers': [32, 32]}
# 0
# 1.4352081556345755 1.5560596697201357
# 1.0941714190696874 0.6090694995879452
# {'epochs': 500, 'dropout': 0.4, 'batch_normalize': False, 'batch_size': 100, 'feat': 'gbnsr6', 'kfold': -1, 'dense_layer_size': 27, 'graph_conv_layers': [32, 32]}
# 0
# 1.8545443800079835 1.6401879965014992
# 1.345571236249819 0.7287521816110063
# {'epochs': 500, 'dropout': 0.4, 'batch_normalize': False, 'batch_size': 100, 'feat': 'igb5', 'kfold': -1, 'dense_layer_size': 27, 'graph_conv_layers': [32, 32]}
# 0
# 3.1124919618370104 2.804571287833655
# 1.5343667447773657 1.0705890607092687
# {'epochs': 500, 'dropout': 0.4, 'batch_normalize': False, 'batch_size': 100, 'feat': 'asc', 'kfold': -1, 'dense_layer_size': 27, 'graph_conv_layers': [32, 32]}
# 0
# 2.6515044880218475 2.489083699259972
# 1.6315918700825154 1.0365083057846594

In [ ]:
#random forest baseline with low overfitting
# tip3p
# physics model: test 1.4352081556345755 train 1.5560596697201357
# physics + ml: test 1.0669119570947632 train 0.751145649425129

# gbnsr6
# physics model: test 1.8545443800079835 train 1.6401879965014992
# physics + ml: test 1.4691780878550593 train 0.7913176545120358

# asc
# physics model: test 2.6515044880218475 train 2.489083699259972
# physics + ml: test 1.499794544503364 train 1.02829254045755

# igb5
# physics model: test 3.1124919618370104 train 2.804571287833655
# physics + ml: test 1.818271652144328 train 1.0540310528007975

# zap9
# physics model: test 1.854544509594727 train 1.6401880873566563
# physics + ml: test 1.4693794976636059 train 0.7914661201123163

# cha
# physics model: test 1.3152918288589681 train 1.7703504244090553
# physics + ml: test 0.9770987137796666 train 0.841078888676981

# null
# physics model: test 5.44957314842181 train 5.4019556726215345
# physics + ml: test 1.7864719621707479 train 1.5013882731855401

# CPU times: user 47.8 s, sys: 814 ms, total: 48.6 s
# Wall time: 9.89 s